In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [61]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [62]:
# Hyperparameters
batch_size = 64
learning_rate = 0.01
num_epochs = 2

In [63]:
transform = transforms.Compose([
    # transforms.Grayscale(),            # Ensure the image is single channel (if not already)
    transforms.ToTensor(),             # Convert image to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [64]:
# import torch.nn as nn
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)  # Output: 8x28x28
        self.pool1 = nn.MaxPool2d(2, 2)  # Output: 8x14x14
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)  # Output: 16x12x12
        self.pool2 = nn.MaxPool2d(2, 2)  # Output: 8x14x14
        self.fc1 = nn.Linear(16 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [65]:
model = SimpleCNN().to(device)

In [66]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [67]:
# 4. Training the model
for epoch in range(num_epochs):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, target)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# 5. Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 95.31%
